# Code
...
# Imports

In [29]:
from pathlib import Path
from datetime import datetime
import pandas as pd
import json 
from sdmetrics.single_column import MissingValueSimilarity, BoundaryAdherence, CategoryAdherence, RangeCoverage, CategoryCoverage
from sdmetrics.reports.single_table import DiagnosticReport, QualityReport
from typing import Optional, Tuple, Union
from scipy.stats import skew, kurtosis

# User Configuration

In [30]:
# Configuration
SAVE_DATA = True  # Whether to save generated figures and CSVs

# File paths
REAL_FILE = "20250301_data_20250510_122405_final_100_train.csv"
SYNTH_FILE = "20250301_data_20250510_122405_final_100_synth.csv"
HOLDOUT_FILE = "20250301_data_20250510_122405_final_100_holdout.csv"
METADATA_FILE = "20250301_data_20250510_122405_final_metadata.json"

# Define directories for input and output
notebook_dir = Path().resolve() # Get the current working directory
DATA_DIR = Path("../../data")
OUTPUT_DIR_CSV = notebook_dir / "results"

# Utility Functions

##  Data Loading

In [31]:
def convert_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Konvertiert bestimmte Spalten des DataFrames in die gewünschten Datentypen:
     - definierte Spalten als category
     - definierte Spalten als pandas Nullable Integer (Int64)
     - consciousness_level und news_score als geordnete Categoricals
    """
    df = df.copy()  # Änderungen nicht am Original vornehmen

    # 1) Kategorische Spalten
    cat_cols = ['gender', 'ethnicity', 'chief_complaint', 'icd_block']
    for col in cat_cols:
        df[col] = df[col].astype('category')

    # 2) Integer-Spalten mit Nullable Integer dtype
    int_cols = ['age', 'systolic_bp', 'diastolic_bp',
                'heart_rate', 'respiratory_rate', 'oxygen_saturation']
    for col in int_cols:
        df[col] = df[col].astype('Int64')

    # 3) Geordnete Categoricals
    df['consciousness_level'] = pd.Categorical(
        df['consciousness_level'],
        categories=['A', 'C', 'V', 'P', 'U'],
        ordered=True
    )
    df['news_score'] = pd.Categorical(
        df['news_score'],
        categories=list(range(19)),
        ordered=True
    )

    return df

def load_data(
    real_filename: Union[str, Path],
    synth_filename: Optional[Union[str, Path]] = None,
    holdout_filename: Optional[Union[str, Path]] = None,
    data_dir: Path = DATA_DIR
) -> Tuple[pd.DataFrame, Optional[pd.DataFrame], Optional[pd.DataFrame]]:
    """
    Lädt die realen, synthetischen und optionalen Holdout-CSV-Dateien
    aus data_dir und wandelt sie über convert_dtypes um.

    Returns:
        df_real: pd.DataFrame
        df_synth: Optional[pd.DataFrame]
        df_holdout: Optional[pd.DataFrame]
    """
    def _read_and_convert(fn: Union[str, Path]) -> pd.DataFrame:
        return (
            pd.read_csv(data_dir / fn, low_memory=False)
              .pipe(convert_dtypes)
        )

    df_real    = _read_and_convert(real_filename)
    df_synth   = _read_and_convert(synth_filename)   if synth_filename   else None
    df_holdout = _read_and_convert(holdout_filename) if holdout_filename else None

    return df_real, df_synth, df_holdout

## Table Stats

In [32]:
def kategorien_uebersicht(df_real, df_synth, kategorien):
    zusammenfassung = []

    for spalte in kategorien:
        real_serie = df_real[spalte].astype('category')
        synth_serie = df_synth[spalte].astype('category')

        real_info = real_serie.describe()
        synth_info = synth_serie.describe()

        zusammenfassung.append({
            'Spalte': spalte,
            'Real_Unique': real_info['unique'],
            'Synth_Unique': synth_info['unique'],
            'Real_Top': real_info['top'],
            'Synth_Top': synth_info['top'],
            'Real_Freq': real_serie.value_counts(normalize=True).iloc[0],
            'Synth_Freq': synth_serie.value_counts(normalize=True).iloc[0],
            # 'Real_Freq': real_info['freq'],
            # 'Synth_Freq': synth_info['freq'],
        })

    return pd.DataFrame(zusammenfassung)

In [33]:
def numerische_uebersicht(df_real, df_synth, numerisch):
    zusammenfassung = []

    for spalte in numerisch:
        real_serie = df_real[spalte].dropna()
        synth_serie = df_synth[spalte].dropna()

        zusammenfassung.append({
            'Spalte': spalte,
            'Real_Median': real_serie.median(),
            'Synth_Median': synth_serie.median(),
            'Real_Q1': real_serie.quantile(0.25),
            'Synth_Q1': synth_serie.quantile(0.25),
            'Real_Q3': real_serie.quantile(0.75),
            'Synth_Q3': synth_serie.quantile(0.75),
            'Real_IQR': real_serie.quantile(0.75) - real_serie.quantile(0.25),
            'Synth_IQR': synth_serie.quantile(0.75) - synth_serie.quantile(0.25),
            'Real_Min': real_serie.min(),
            'Synth_Min': synth_serie.min(),
            'Real_Max': real_serie.max(),
            'Synth_Max': synth_serie.max(),
            'Real_Skew': skew(real_serie),
            'Synth_Skew': skew(synth_serie),
            'Real_Kurtosis': kurtosis(real_serie),  # excess kurtosis
            'Synth_Kurtosis': kurtosis(synth_serie),
        })

    return pd.DataFrame(zusammenfassung)


# Main Routine

In [34]:
OUTPUT_DIR_CSV.mkdir(parents=True, exist_ok=True)  # Ensure output dir exists

with open(DATA_DIR / METADATA_FILE, 'r') as f:
    my_metadata_dict = json.load(f)

df_real, df_synth, df_holdout = load_data(REAL_FILE, SYNTH_FILE, HOLDOUT_FILE)

ts = datetime.now().strftime("%Y%m%d_%H%M%S")

## Check Missing Values (Redundant, as Filtered Out Prior CTGAN)

In [35]:
df_missing_real = df_real[df_real.isna().any(axis=1)].reset_index(drop=True)
print(df_missing_real.shape)

df_missing_synth = df_synth[df_synth.isna().any(axis=1)].reset_index(drop=True)
print(df_missing_synth.shape)

(0, 16)
(0, 16)


In [36]:
results = [
    MissingValueSimilarity.compute(
        real_data=df_real[col],
        synthetic_data=df_synth[col]
    )
    for col in df_real.columns
]

print(results)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


## Category and Boundary Adherence

In [37]:
report = DiagnosticReport()
report.generate(df_real, df_synth, my_metadata_dict)
print("--------------------------")
print(f"Get Properties: {report.get_properties()}")
print("--------------------------")
print(f"Get Score: {report.get_score()}")
print("--------------------------")
print(f"Get Details, Data Structure: {report.get_details(property_name='Data Structure')}")

Generating report ...

(1/2) Evaluating Data Validity: |██████████| 16/16 [00:00<00:00, 347.72it/s]|
Data Validity Score: 99.93%

(2/2) Evaluating Data Structure: |██████████| 1/1 [00:00<00:00, 1001.74it/s]|
Data Structure Score: 100.0%

Overall Score (Average): 99.96%

--------------------------
Get Properties:          Property     Score
0   Data Validity  0.999267
1  Data Structure  1.000000
--------------------------
Get Score: 0.9996335467584156
--------------------------
Get Details, Data Structure:            Metric  Score
0  TableStructure    1.0


In [38]:
export = report.get_details(property_name='Data Validity')
print(export)
# if SAVE_DATA:
    # export.to_csv(OUTPUT_DIR_CSV / f'adherence_{ts}.csv', index=False, sep=';', decimal=',', encoding='utf-8')

                 Column             Metric     Score
0     icu_admission_24h  CategoryAdherence  1.000000
1                   age  BoundaryAdherence  0.988273
2                gender  CategoryAdherence  1.000000
3             ethnicity  CategoryAdherence  1.000000
4   consciousness_level  CategoryAdherence  1.000000
5           temperature  BoundaryAdherence  1.000000
6            heart_rate  BoundaryAdherence  1.000000
7      respiratory_rate  BoundaryAdherence  1.000000
8     oxygen_saturation  BoundaryAdherence  1.000000
9           systolic_bp  BoundaryAdherence  1.000000
10         diastolic_bp  BoundaryAdherence  1.000000
11           news_score  CategoryAdherence  1.000000
12        night_arrival  CategoryAdherence  1.000000
13      weekend_arrival  CategoryAdherence  1.000000
14      chief_complaint  CategoryAdherence  1.000000
15            icd_block  CategoryAdherence  1.000000


## Category and Range Coverage

In [39]:
num_attrs = df_real.select_dtypes(include=['number']).columns
cat_attrs = df_real.select_dtypes(exclude=['number']).columns

# compute all scores up front
num_scores = {
    attr: RangeCoverage.compute(
        real_data=df_real[attr], synthetic_data=df_synth[attr]
    )
    for attr in num_attrs
}
cat_scores = {
    attr: CategoryCoverage.compute(
        real_data=df_real[attr], synthetic_data=df_synth[attr]
    )
    for attr in cat_attrs
}

# build the result rows in the same column order as df_real
rows = []
for col in df_real.columns:
    if col in num_scores:
        rows.append({
            'Column': col,
            'Metric': 'RangeCoverage',
            'Score': num_scores[col]
        })
    elif col in cat_scores:
        rows.append({
            'Column': col,
            'Metric': 'CategoryCoverage',
            'Score': cat_scores[col]
        })
    else:
        # skip any extra columns
        continue

# assemble into a DataFrame and print
result_df = pd.DataFrame(rows)
print(result_df)
# if SAVE_DATA:
    # result_df.to_csv(OUTPUT_DIR_CSV / f'coverage_{ts}.csv', index=False, sep=';', decimal=',', encoding='utf-8')

                 Column            Metric     Score
0     icu_admission_24h  CategoryCoverage  1.000000
1                   age     RangeCoverage  1.000000
2                gender  CategoryCoverage  1.000000
3             ethnicity  CategoryCoverage  1.000000
4   consciousness_level  CategoryCoverage  1.000000
5           temperature     RangeCoverage  0.397163
6            heart_rate     RangeCoverage  0.637681
7      respiratory_rate     RangeCoverage  0.372881
8     oxygen_saturation     RangeCoverage  0.260000
9           systolic_bp     RangeCoverage  0.736607
10         diastolic_bp     RangeCoverage  0.872000
11           news_score  CategoryCoverage  1.000000
12        night_arrival  CategoryCoverage  1.000000
13      weekend_arrival  CategoryCoverage  1.000000
14      chief_complaint  CategoryCoverage  1.000000
15            icd_block  CategoryCoverage  1.000000


## Stats Tables

In [40]:
stats_cats = kategorien_uebersicht(df_real, df_synth, cat_attrs)
stats_cats = stats_cats.merge(result_df[['Column', 'Score']].rename(columns={'Column': 'Spalte'}), on='Spalte', how='left')
stats_cats = stats_cats.rename(columns={'Score': 'Coverage'})
stats_cats = stats_cats.merge(export[['Column', 'Score']].rename(columns={'Column': 'Spalte'}), on='Spalte', how='left')
stats_cats = stats_cats.rename(columns={'Score': 'Adherence'})
print(stats_cats)
if SAVE_DATA:
    stats_cats.to_csv(OUTPUT_DIR_CSV / f'cat_stats_{ts}.csv', index=False, sep=';', decimal=',', encoding='utf-8')

                Spalte  Real_Unique  Synth_Unique Real_Top Synth_Top  \
0    icu_admission_24h            2             2    False     False   
1               gender            2             2        F         F   
2            ethnicity            5             5    White     White   
3  consciousness_level            5             4        A         A   
4           news_score           14            10        0         0   
5        night_arrival            2             2    False     False   
6      weekend_arrival            2             2    False     False   
7      chief_complaint          309           309    Other     Other   
8            icd_block          219           219  I30-I5A   A30-A49   

   Real_Freq  Synth_Freq  Coverage  Adherence  
0   0.876739    0.906392       1.0        1.0  
1   0.514653    0.589623       1.0        1.0  
2   0.680677    0.624589       1.0        1.0  
3   0.919055    0.967345       1.0        1.0  
4   0.362962    0.369171       1.0     

In [41]:
stats_num = numerische_uebersicht(df_real, df_synth, num_attrs)
stats_num = stats_num.merge(result_df[['Column', 'Score']].rename(columns={'Column': 'Spalte'}), on='Spalte', how='left')
stats_num = stats_num.rename(columns={'Score': 'Coverage'})
stats_num = stats_num.merge(export[['Column', 'Score']].rename(columns={'Column': 'Spalte'}), on='Spalte', how='left')
stats_num = stats_num.rename(columns={'Score': 'Adherence'})
print(stats_num)
if SAVE_DATA:
    stats_num.to_csv(OUTPUT_DIR_CSV / f'num_stats_{ts}.csv', index=False, sep=';', decimal=',', encoding='utf-8')

              Spalte  Real_Median  Synth_Median  Real_Q1  Synth_Q1  Real_Q3  \
0                age         61.0          61.0     47.0      52.0     74.0   
1        temperature         36.7          36.7     36.4      36.4     37.0   
2         heart_rate         85.0          81.0     73.0      75.0     99.0   
3   respiratory_rate         18.0          18.0     16.0      16.0     18.0   
4  oxygen_saturation         98.0          99.0     97.0      98.0    100.0   
5        systolic_bp        132.0         137.0    117.0     117.0    149.0   
6       diastolic_bp         75.0          75.0     64.0      64.0     85.0   

   Synth_Q3  Real_IQR  Synth_IQR  Real_Min  Synth_Min  Real_Max  Synth_Max  \
0      75.0      27.0       23.0      18.0       18.0      91.0       95.0   
1      36.9       0.6        0.5      30.0       35.0      44.1       40.6   
2      98.0      26.0       23.0      20.0       40.0     227.0      172.0   
3      18.0       2.0        2.0       1.0       13.0  